In [ ]:
!curl localhost:9200

#pip install -q tqdm notebook==7.1.2 openai elasticsearch pandas scikit-learn transformers

In [ ]:
import requests

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

print(documents)

In [ ]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')

es_client.info()

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

documents[0]

In [ ]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

In [ ]:
def elastic_search(query, course='data-engineering-zoomcamp', size=3):
    search_query = {
        "size": size,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    result_docs = [hit['_source'] for hit in response['hits']['hits']]
    return result_docs, response['hits']['max_score']

In [ ]:
query = "How do I execute a command in a running docker container?"
result_docs, max_score = elastic_search(query)
print(f"The score for the top ranking result is: {max_score:.2f}")

query = "How do I execute a command in a running docker container?"
result_docs, _ = elastic_search(query, course='machine-learning-zoomcamp')
third_question = result_docs[2]['question'] if len(result_docs) > 2 else None
print(f"The 3rd question returned by the search engine is: {third_question}")

In [ ]:
context_template = """
Q: {question}
A: {text}
""".strip()

context = "\n\n".join(context_template.format(**doc) for doc in result_docs)

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

prompt = prompt_template.format(question=query, context=context)

prompt_length = len(prompt)
print(f"The length of the resulting prompt is: {prompt_length}")

In [ ]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")
encoded_prompt = encod